In [2]:
import pandas as pd
from crawlers.crawler import Crawler
from bs4 import BeautifulSoup
from base import BaseClass
from common.text_normalizer import normalized
import json
import requests
import os
from pprint import pprint
from schema.validate import ValidateUsedCars

                       name  \
0     Mercedes-Benz C-class   
1     Mercedes-Benz C-class   
2               KIA Sorento   
3     Mercedes-Benz S-class   
4                  Lexus RX   
5                   Mazda 2   
6               Kia Morning   
7   Mercedes-Benz GLA-Class   
8         Chevrolet Captiva   
9               Ford Ranger   
10                Kia Forte   
11                 Lexus RX   
12                  KIA Rio   
13             Toyota Altis   
14                Kia Forte   
15                  Audi Q3   
16           Nissan X-Trail   
17             Ford Everest   
18           Hyundai Starex   
19           Hyundai Sonata   

                                                  url    origin   km_driven  \
0   https://www.sanxehot.vn//mua-ban-xe/mercedes--...  domestic           0   
1   https://www.sanxehot.vn//mua-ban-xe/mercedes--...  domestic           0   
2   https://www.sanxehot.vn//mua-ban-xe/kia-new-so...    import   49.000 km   
3   https://www.sanxehot.vn//mua

In [52]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; rv:46.0) "
                  "Gecko/20100101 Firefox/46.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Connection": "keep-alive",
    "Cache-Control": "max-age=0"
}


class SanXeHotCrawler():
    def __init__(self):
        self.path = '/Users/minhkhoa/Documents/used-cars-prices-prediction/crawlers/'
        self.link_list = open(self.path + 'test_file1.txt', 'r').readlines()

    def crawl(self):
        car_df = pd.DataFrame(columns=['name', 'url', 'origin', 'km_driven', 'external_color',
                                       'internal_color', 'seats', 'fuels', 'engine_capacity',
                                       'transmission', 'wheel_drive', 'price', 'year'])
        num = 0
        for link in self.link_list:
            num += 1
            url = link.strip()

            req = requests.get(url, headers=HEADERS).text
            soup = BeautifulSoup(req, 'lxml')
            body_page = soup.find('table', class_='info').tbody
            car_feature = body_page.find_all('td')

            gia = car_feature[0].text.strip()
            gia = gia.replace(',', '')
            if 'triệu' in gia:
                price = gia.replace(' triệu', '000000')
            elif 'tỷ' in gia:
                price = gia.replace(' tỷ', '000000000')

            name = car_feature[2].text.strip()
            nam = car_feature[4].text.strip().split()
            if len(nam) > 1:
                year = nam[-1]
            else:
                year = nam[0]

            km_driven = car_feature[6].text.strip().split()[0]
            km_driven = km_driven.replace('.', '')

            dong_co = car_feature[8].text.strip().split()
            if len(dong_co) == 2:
                fuels = dong_co[0]
                engine_capacity = dong_co[1].split('L')[0]
            else:
                fuels = dong_co[0]
                engine_capacity = None

            hop_so = car_feature[10].text.strip()
            if hop_so == 'Số sàn':
                transmission = 'manual'
            elif hop_so == 'Tự động':
                transmission = 'automatic'

            xuat_xu = car_feature[12].text.strip()
            if 'trong nước' in xuat_xu:
                origin = 'import'
            elif 'Nhập khẩu' in xuat_xu:
                origin = 'domestic'
            else:
                origin = None

            external_color = car_feature[14].text.strip()
            internal_color = None
            seats = None
            wheel_drive = None
            print(num)
            car_df = car_df.append(pd.Series({'name': name, 'url': link, 'origin': origin, 'km_driven': km_driven,
                                              'external_color': external_color, 'internal_color': internal_color,
                                              'seats': seats, 'fuels': fuels, 'engine_capacity': engine_capacity,
                                              'transmission': transmission, 'wheel_drive': wheel_drive,
                                              'price': price, 'year': year}), ignore_index=True)
        return car_df

In [53]:
var = SanXeHotCrawler()
df = var.crawl()
df

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200


,name,url,origin,km_driven,external_color,internal_color,seats,fuels,engine_capacity,transmission,wheel_drive,price,year
0,Mercedes-Benz C-class,https://www.sanxehot.vn//mua-ban-xe/mercedes--...,domestic,0,Ghi xám,None,None,Xăng,None,manual,None,55000000,1980
1,Mercedes-Benz C-class,https://www.sanxehot.vn//mua-ban-xe/mercedes--...,domestic,0,Ghi xám,None,None,Xăng,None,manual,None,55000000,1980
2,KIA Sorento,https://www.sanxehot.vn//mua-ban-xe/kia-new-so...,import,49000,Bạc,None,None,Xăng,2.2,automatic,None,810000000,2014
3,Mercedes-Benz S-class,https://www.sanxehot.vn//mua-ban-xe/mercedes--...,import,20000,Đen,None,None,Xăng,3.0,automatic,None,3790000000000,2016
4,Lexus RX,https://www.sanxehot.vn//mua-ban-xe/lexus-rx35...,domestic,30000,Trắng,None,None,Xăng,3.5,automatic,None,2190000000000,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Ford Everest,https://www.sanxehot.vn//mua-ban-xe/ford-evere...,import,85000,Đen,None,None,Diesel,2.5,manual,None,515000000,2009
196,Ford Ranger,https://www.sanxehot.vn//mua-ban-xe/ford-range...,domestic,76000,Trắng,None,None,Xăng,2.2,manual,None,470000000,2012
197,Ford Escape,https://www.sanxehot.vn//mua-ban-xe/ford-escap...,import,77000,Bạc,None,None,Xăng,2.3,automatic,None,528000000,2010
198,Ford Everest,https://www.sanxehot.vn//mua-ban-xe/ford-evere...,import,28000,Bạc,None,None,Diesel,2.5,automatic,None,790000000,2014
